# UnixCoder Demo

In [ ]:
%%capture

!pip install datasets==3.6.0

In [ ]:
!wget https://raw.githubusercontent.com/microsoft/CodeBERT/master/UniXcoder/unixcoder.py

--2025-08-03 00:06:32--  https://raw.githubusercontent.com/microsoft/CodeBERT/master/UniXcoder/unixcoder.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10352 (10K) [text/plain]
Saving to: ‘unixcoder.py’

unixcoder.py        100%[===================>]  10.11K  --.-KB/s    in 0s      

2025-08-03 00:06:32 (35.5 MB/s) - ‘unixcoder.py’ saved [10352/10352]



In [ ]:
from transformers import (
    AutoModel,
    AutoTokenizer
)
import torch
from datasets import load_dataset
from unixcoder import UniXcoder

In [ ]:
code_dataset = load_dataset("code-search-net/code_search_net", "python");

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

code_search_net.py: 0.00B [00:00, ?B/s]

The repository for code-search-net/code_search_net contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/code-search-net/code_search_net.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


python.zip:   0%|          | 0.00/941M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/412178 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/22176 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/23107 [00:00<?, ? examples/s]

In [ ]:
train_data = code_dataset["train"]
train_data

Dataset({
    features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
    num_rows: 412178
})

In [ ]:
# Experiment, randomly select 1 out of the first 50 entries in train_data
import random
rand_int1 = random.randint(0, 50)
rand_int2 = random.randint(0, 50)
while rand_int2 == rand_int1:
    rand_int2 = rand_int1

In [ ]:
sample_1 = train_data[rand_int1]['func_code_string']
sample_2 = train_data[rand_int2]['func_code_string']
print(f"{sample_1=}")
print(f"{sample_2=}")

sample_1='def resolve_absolute_name(self, name):\n        \'\'\'\n        Resolve a field from an absolute name.\n        An absolute name is just like unix absolute path,\n        starts with \'/\' and each name component is separated by \'/\'.\n\n        :param name: absolute name, e.g. "/container/subcontainer/field"\n        :return: field with this absolute name\n        :raises: KittyException if field could not be resolved\n        \'\'\'\n        current = self\n        while current.enclosing:\n            current = current.enclosing\n        if name != \'/\':\n            components = name.split(\'/\')[1:]\n            for component in components:\n                current = current.get_field_by_name(component)\n        return current'
sample_2="def hash(self):\n        '''\n        :rtype: int\n        :return: hash of the field\n        '''\n        hashed = super(Calculated, self).hash()\n        return khash(hashed, self._field_name)"


In [ ]:
def clean_code_snippet(code_snippet: str) -> str:
    cleaned = code_snippet.replace("\n", "").replace("\t", " ")
    cleaned = cleaned.replace("    ", " ")
    tripple_quotes = '"""' if cleaned.find('"""') != -1 else "'''"
    l = cleaned.find(tripple_quotes)
    r = cleaned.find(tripple_quotes, l + 3) + 3
    cleaned = cleaned[:l] + cleaned[r:]
    return cleaned.replace("    ", " ")


In [ ]:
cleaned_sample_1 = clean_code_snippet(sample_1)
cleaned_sample_2 = clean_code_snippet(sample_2)
print(f"{cleaned_sample_1=}")
print(f"{cleaned_sample_2=}")

cleaned_sample_1="def resolve_absolute_name(self, name): current = self  while current.enclosing:   current = current.enclosing  if name != '/':   components = name.split('/')[1:]   for component in components: current = current.get_field_by_name(component)  return current"
cleaned_sample_2='def hash(self): hashed = super(Calculated, self).hash()  return khash(hashed, self._field_name)'


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = UniXcoder("microsoft/unixcoder-base")
model.to(device)

cpu


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/691 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/504M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/504M [00:00<?, ?B/s]

UniXcoder(
  (model): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(51416, 768, padding_idx=1)
      (position_embeddings): Embedding(1026, 768, padding_idx=1)
      (token_type_embeddings): Embedding(10, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): Layer

In [ ]:
func = cleaned_sample_1
tokens_ids = model.tokenize([func], max_length=512, mode="<encoder-only>")
source_ids = torch.tensor(tokens_ids).to(device)
tokens_embeddings, set_class_embeddings = model(source_ids)

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.58.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [ ]:
func = cleaned_sample_2
tokens_ids = model.tokenize([func], max_length=512, mode="<encoder-only>")
source_ids = torch.tensor(tokens_ids).to(device)
tokens_embeddings, video_func_embeddings = model(source_ids)

In [ ]:
tokens_embeddings.shape

torch.Size([1, 8, 768])

In [ ]:
# a natural description
nl = "returns a video placeholder"
tokens_ids = model.tokenize([nl],max_length=512,mode="<encoder-only>")
source_ids = torch.tensor(tokens_ids).to(device)
tokens_embeddings,nl_embedding = model(source_ids)

In [ ]:
video_func_embeddings.shape
s = str(video_func_embeddings.tolist())
# convert back to list then tensor
l = eval(s)
t = torch.tensor(l)
print(t.shape)

torch.Size([1, 768])


In [ ]:
nl_embedding.shape

torch.Size([1, 768])

In [ ]:
set_class_embd  = torch.nn.functional.normalize(set_class_embeddings, p=2, dim=1)
video_func_embd = torch.nn.functional.normalize(video_func_embeddings, p=2, dim=1)
nl_embd         = torch.nn.functional.normalize(nl_embedding, p=2, dim=1)

In [ ]:
sim1 = torch.einsum("ac,bc->ab", set_class_embd, nl_embd)
sim2 = torch.einsum("ac,bc->ab", video_func_embd, nl_embd)

print(sim1)
print(sim2)

tensor([[0.0727]], grad_fn=<ViewBackward0>)
tensor([[0.1072]], grad_fn=<ViewBackward0>)


# Constructing JSON

In [7]:
!unzip KG4Py.zip
!chmod +x KG4Py/script/run.sh

Archive:  KG4Py.zip
   creating: KG4Py/
  inflating: KG4Py/LICENSE.md        
  inflating: KG4Py/test_output.json  
   creating: KG4Py/demo_repo/
  inflating: KG4Py/demo_repo/another.txt  
  inflating: KG4Py/demo_repo/test.py  
  inflating: KG4Py/.DS_Store         
   creating: KG4Py/finetuning_sbert/
  inflating: KG4Py/finetuning_sbert/QGPipeline.py  
  inflating: KG4Py/finetuning_sbert/ModelTraning.py  
  inflating: KG4Py/finetuning_sbert/CodeSearchNet.ipynb  
  inflating: KG4Py/finetuning_sbert/DocstringDatasetProcesser.py  
  inflating: KG4Py/uv.lock           
 extracting: KG4Py/data.zip          
  inflating: KG4Py/pyproject.toml    
   creating: KG4Py/tests/
   creating: KG4Py/manim/
   creating: KG4Py/manim/renderer/
  inflating: KG4Py/manim/renderer/cairo_renderer.py  
  inflating: KG4Py/manim/renderer/shader.py  
  inflating: KG4Py/manim/renderer/opengl_renderer_window.py  
 extracting: KG4Py/manim/renderer/__init__.py  
  inflating: KG4Py/manim/renderer/vectorized_mobject_re

In [8]:
!pip install libcst

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 24.8 MB/s eta 0:00:00


In [ ]:
!cd KG4Py/ && pwd && ./script/run.sh

/content/KG4Py
Running feature extraction /content/KG4Py/manim
2025-08-08 06:46:54,367 - INFO - Identifying source files in /content/KG4Py/manim.
2025-08-08 06:46:54,367 - INFO - Found source file: /content/KG4Py/manim/constants.py
2025-08-08 06:46:54,367 - INFO - Found source file: /content/KG4Py/manim/typing.py
2025-08-08 06:46:54,367 - INFO - Found source file: /content/KG4Py/manim/__init__.py
2025-08-08 06:46:54,367 - INFO - Found source file: /content/KG4Py/manim/__main__.py
2025-08-08 06:46:54,367 - INFO - Found source file: /content/KG4Py/manim/utils/caching.py
2025-08-08 06:46:54,368 - INFO - Found source file: /content/KG4Py/manim/utils/deprecation.py
2025-08-08 06:46:54,368 - INFO - Found source file: /content/KG4Py/manim/utils/polylabel.py
2025-08-08 06:46:54,368 - INFO - Found source file: /content/KG4Py/manim/utils/tex_templates.py
2025-08-08 06:46:54,368 - INFO - Found source file: /content/KG4Py/manim/utils/opengl.py
2025-08-08 06:46:54,368 - INFO - Found source file: /c

In [ ]:
import os
os.listdir()

['.config', 'KG4Py', 'KG4Py.zip', 'sample_data']

In [ ]:
import json

with open("KG4Py/data/manim_features.json", "r") as f:
    data = json.load(f)

print(f"Processed modules: {len(data)}")

Processed modules: 162


In [ ]:
import random

sample = random.choice(data)
sample

{'name': 'autocolor_directive',
 'docstring': 'A directive for documenting colors in Manim.',
 'imports': ['__future__.annotations',
  'inspect',
  'typing.TYPE_CHECKING',
  'docutils.nodes',
  'docutils.parsers.rst.Directive',
  'manim.ManimColor',
  'sphinx.application.Sphinx',
  'importlib'],
 'classes': [{'name': 'ManimColorModuleDocumenter',
   'docstring': '',
   'embedding': [[-0.3412245512008667,
     0.03229442983865738,
     0.23538289964199066,
     -1.5612670183181763,
     0.2228851467370987,
     -0.644387423992157,
     -0.6740632653236389,
     -0.040024835616350174,
     1.4021413326263428,
     -0.48790231347084045,
     -0.25784534215927124,
     -1.7604401111602783,
     -0.6618919372558594,
     -1.2400922775268555,
     -0.16935259103775024,
     0.6178722381591797,
     -0.09530490636825562,
     2.994349718093872,
     -1.537255883216858,
     0.901665985584259,
     0.31796422600746155,
     0.9385297894477844,
     1.5373796224594116,
     -0.10312385857105255

In [ ]:
!zip -r data.zip KG4Py/data/

  adding: KG4Py/data/ (stored 0%)
  adding: KG4Py/data/manim_features.json (deflated 55%)
  adding: KG4Py/data/failed_operations/ (stored 0%)
  adding: KG4Py/data/failed_operations/tex_failed.json (deflated 77%)
  adding: KG4Py/data/failed_operations/__init___failed.json (deflated 25%)
  adding: KG4Py/data/failed_operations/DVIPSNAMES_failed.json (deflated 43%)
  adding: KG4Py/data/graph_schema.json (deflated 83%)


# Build Chroma db

In [9]:
%%capture

!pip install chromadb

In [44]:
!pwd
!cd KG4Py/ && pwd && ./script/run.sh

/content
/content/KG4Py
Running feature extraction /content/KG4Py/demo_repo
Processing modules: 100% 162/162 [00:00<00:00, 225814.97it/s]
Adding items to chromadb: 100% 724/724 [00:24<00:00, 29.14it/s]
Chromadb database created at /content/KG4Py/data/chromadb


In [45]:
import chromadb

collection_name = "manim_embeddings"
path = "./KG4Py/data/chromadb"

client = chromadb.PersistentClient(path=path)
for collection in client.list_collections():
    print(collection.name)

manim_embeddings


# Testing chromadb

In [10]:
!wget https://raw.githubusercontent.com/microsoft/CodeBERT/master/UniXcoder/unixcoder.py

--2025-08-11 07:07:51--  https://raw.githubusercontent.com/microsoft/CodeBERT/master/UniXcoder/unixcoder.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10352 (10K) [text/plain]
Saving to: ‘unixcoder.py’

unixcoder.py        100%[===================>]  10.11K  --.-KB/s    in 0.001s  

2025-08-11 07:07:51 (15.0 MB/s) - ‘unixcoder.py’ saved [10352/10352]



In [54]:
import chromadb
from chromadb.config import Settings
from chromadb import Documents
from unixcoder import UniXcoder
import torch


class ChromaDBQueryEngine:
    def __init__(self, collection_name, persist_directory, limit=5):
        """
        Initialize the ChromaDB query engine.

        Args:
            collection_name (str): The name of the collection to query.
            persist_directory (str): Directory where the ChromaDB database is stored.
            limit (int): Maximum number of results to return.
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.chroma_client = chromadb.PersistentClient(
                path=persist_directory,
            )
        self.collection = self.chroma_client.get_collection(name=collection_name)
        print("Collection name:", collection.name)

        self.limit = limit
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = UniXcoder("microsoft/unixcoder-base")
        self.model.to(self.device)

    def embedd_query(self, query: str) -> list[float]:
        tokens_id = self.model.tokenize(
            [f"{query}"], max_length=512, mode="<encoder-only>"
        )
        source_id = torch.tensor(tokens_id, device=self.device)
        _, embedding = self.model(source_id)
        embedding = embedding.tolist()
        return embedding

    def execute_query_nl(self, query: str) -> list[list[Documents]]:
        embedding = self.embedd_query(query)
        result = self.collection.query(query_embeddings=embedding, n_results=self.limit)
        return result

    def execute_query_e(self, embedding: list[float]):
        pass


In [55]:
engine = ChromaDBQueryEngine("manim_embeddings", "./KG4Py/data/chromadb")

Collection name: manim_embeddings


In [62]:
result = engine.execute_query_nl("Zoom")
result

{'ids': [['215', '234', '237', '329', '350']],
 'embeddings': None,
 'documents': [[None, None, None, None, None]],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'type': 'function', 'name': 'index_labels'},
   {'name': 'result', 'type': 'function'},
   {'name': 'result', 'type': 'function'},
   {'type': 'class', 'name': 'ManimFrame'},
   {'name': 'VectorScene', 'type': 'class'}]],
 'distances': [[867.190673828125,
   884.5103759765625,
   884.5103759765625,
   923.7503662109375,
   923.7503662109375]]}